In [1]:
# Importing necessary libraries
from gensim.models import TfidfModel
import numpy as np
import json
import glob

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# #spacy
import spacy
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#helper functions for data writing/reading
def load_data(file):
    # Function to load data from a JSON file
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def write_data(file, data):
    # Function to write data to a JSON file
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [3]:
#helper functions for lda preprocessing

stopwords = stopwords.words("english") + ["go", "re", "just", "get", "want", "know", "so", "need", "knock", "look", "guy", "work", "say", "let", "come", "here", "make", "see",  "tell", "thing", "talk", "ve", "really"]


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    # Function for lemmatizing the text and removing stopwords
    """https://spacy.io/api/annotation"""
    nlp = spacy.load(
        "/Users/anishpalakurthi/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-3.6.0", disable=["parser", "ner"])
    texts_out = []
    for sent in texts:
        # Lemmatizes each word by appending allowed tokens from the doc object's metadata
        doc = nlp(sent)
        texts_out.append(" ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))

    # Removing stopwords from the lemmatized texts
    for word in texts_out:
        if word in stopwords:
            texts_out.remove(word)
    return texts_out


def gen_words(texts):
    # Function to preprocess the lemmatized keywords
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

In [4]:
#handle bigrams and trigrams
def make_bigrams(texts):
    # Applying bigram models to the data_words
    bigram_phrases = gensim.models.Phrases(texts, min_count=5, threshold=50)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    return [bigram[doc] for doc in texts]


def make_trigrams(texts):
    # Applying trigram models to the data_words
    bigram_phrases = gensim.models.Phrases(texts, min_count=5, threshold=50)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[texts], threshold=50)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)
    return [trigram[bigram[doc]] for doc in texts]


In [5]:
# Load and preprocess data with helper calls
episodeData = load_data("data/cleaned.json")
plots = [episodeData[episode] for episode in episodeData]
lemmatized_texts = lemmatization(plots)
data_words = gen_words(lemmatized_texts)


In [6]:
# Create bigrams and trigrams
data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

In [7]:
# Applying TF-IDF removal to the corpus
id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

In [8]:
low_value = 0.03
words = []
words_missing_in_tfidf = []


# Identifying words to be removed based on their TF-IDF values
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]

    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])

    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [9]:
# Reconstructing the corpus after TF-IDF removal
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]


In [10]:
# Generating the LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=5,
                                            random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Enabling the notebook for visualization
pyLDAvis.enable_notebook()

# Preparing the data for visualization using pyLDAvis
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)

# Displaying the visualization
vis

/Users/anishpalakurthi/Library/Python/3.10/lib/python/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.166613  0.219743       1        1  60.491258
3     -0.179513  0.128088       2        1  36.154792
1      0.046303 -0.166944       3        1   1.849100
2      0.139684 -0.119061       4        1   0.886001
0      0.160139 -0.061826       5        1   0.618848, topic_info=           Term        Freq       Total Category  logprob  loglift
1227      knock  546.000000  546.000000  Default  30.0000  30.0000
382       penny  624.000000  624.000000  Default  29.0000  29.0000
143        door  286.000000  286.000000  Default  28.0000  28.0000
3380       spot  102.000000  102.000000  Default  27.0000  27.0000
3709   birthday  109.000000  109.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
3419         aw    1.185832    7.111624   Topic5  -7.0899   3.2938
1917   approach    1.091697   22.869531   Topic5  -7.1726   2.0430
1266      olive    0.990285    4.733859   Topic5  -7.2701   3.5206
1114  chocolate    1.066702   40.995039   Topic5  -7.1958   1.4362
1315    robotic    0.955398    2.929729   Topic5  -7.3060   3.9645

[375 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
6978      3  0.429046       abort
6978      4  0.429046       abort
3         1  0.990101     actress
1904      3  0.791147  adolescent
8         1  0.387439       agree
...     ...       ...         ...
580       2  0.467303        work
2896      1  0.980046    workshop
3407      1  0.955132        your
3407      2  0.037456        your
5993      4  0.620540        zoom

[586 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])

In [22]:

def find_closest_episodes(user_input_episode, lda_model, episode_data, num_similar=5):
    # Check if the user input is a valid episode number
    if user_input_episode.isdigit() and int(user_input_episode) <= len(episode_data):
        episode_number = int(user_input_episode)
        target_episode = episode_data[episode_number - 1]
    else:
        # Assuming the user input is the title of an episode
        target_episode = user_input_episode

    # Get the topic distribution vector for the target episode
    target_corpus = id2word.doc2bow(simple_preprocess(target_episode, deacc=True))
    target_topics = lda_model.get_document_topics(target_corpus, minimum_probability=0)

    similarity_scores = []
    for episode_num, episode_corpus in enumerate(episode_data):
        doc_corpus = id2word.doc2bow(simple_preprocess(episode_corpus, deacc=True))
        doc_topics = lda_model.get_document_topics(doc_corpus, minimum_probability=0)
        similarity_score = gensim.matutils.cossim(target_topics, doc_topics)
        similarity_scores.append((episode_num + 1, similarity_score))

    # Sort episodes based on similarity scores (descending order)
    sorted_episodes = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get top N similar episodes
    top_similar_episodes = sorted_episodes[:num_similar]

    return top_similar_episodes



# User input: enter an episode title
user_input_episode = input("Enter the title of an episode: ")

# Find closest similarity episodes using LDA model
num_similar_episodes = 5
closest_episodes = find_closest_episodes(user_input_episode, lda_model, plots, num_similar_episodes)

# Display the closest episodes
print(f"\nTop {num_similar_episodes} similar episodes based on LDA topic distribution:")
for episode, similarity_score in closest_episodes:
    print(f"Episode: {episode}, Similarity Score: {similarity_score:.4f}")









Top 5 similar episodes based on LDA topic distribution:
Episode: 2, Similarity Score: 1.0000
Episode: 71, Similarity Score: 0.9992
Episode: 26, Similarity Score: 0.9988
Episode: 74, Similarity Score: 0.9980
Episode: 20, Similarity Score: 0.9973
